In [1]:
import pandas as pd

# Carregar os dados
base_codificada_path = "../Bases/Criados/5_base_codificada.csv"
dicionario_path = "../Bases/Originais/PNS_titulos.csv"

def processar_dicionario(filepath):
    """Processa o arquivo de dicionário que está mal formatado"""
    with open(filepath, 'r', encoding='utf-8') as f:
        linhas = f.readlines()
    
    dicionario = {}
    for linha in linhas:
        if ';' in linha:  # Linhas que contêm definições
            partes = linha.strip().split(';')
            if len(partes) >= 2:
                var = partes[0].strip()
                desc = ';'.join(partes[1:]).strip()
                dicionario[var] = desc
    return dicionario

try:
    # Processar os arquivos
    df = pd.read_csv(base_codificada_path)
    dicionario = processar_dicionario(dicionario_path)
    
    # Criar tabela de resultados
    resultados = []
    
    for coluna in df.columns:
        # Buscar descrição no dicionário
        descricao = dicionario.get(coluna, f"Atributo Criado: {coluna}")
        
        # Obter valores únicos (tratando missing values)
        valores_unicos = df[coluna].dropna().astype(str).unique()
        valores_unicos = [v for v in valores_unicos if v not in ['nan', '']]
        
        # Formatar exibição
        limite = 5
        if len(valores_unicos) > limite:
            valores_str = ', '.join(valores_unicos[:limite]) + f'... (+{len(valores_unicos)-limite} mais)'
        else:
            valores_str = ', '.join(valores_unicos) if valores_unicos else "Sem valores"
        
        resultados.append({
            'Atributo': coluna,
            'Descrição': descricao,
            'Valores Únicos': valores_str,
            'Qtd Valores Únicos': len(valores_unicos)
        })
    
    # Criar e salvar DataFrame
    tabela_final = pd.DataFrame(resultados)
    output_path = "../Bases/Criados/resumo_atributos.csv"
    tabela_final.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
    
    # Gerar relatório
    print("PROCESSAMENTO CONCLUÍDO COM SUCESSO")
    print("="*50)
    print(f"Total de atributos processados: {len(df.columns)}")
    print(f"Atributos com descrição encontrada: {len([d for d in resultados if not d['Descrição'].startswith('Atributo Criado')])}")
    print(f"Atributos criados/sem descrição: {len([d for d in resultados if d['Descrição'].startswith('Atributo Criado')])}")
    print("\nPrimeiras linhas do resultado:")
    print(tabela_final.head().to_string(index=False))
    print(f"\nArquivo salvo em: {output_path}")

except Exception as e:
    print("\nERRO DURANTE A EXECUÇÃO")
    print("="*50)
    print(f"Tipo do erro: {type(e).__name__}")
    print(f"Mensagem: {str(e)}")
    print("\nSugestões:")
    print("- Verifique se os caminhos dos arquivos estão corretos")
    print("- Confira se o arquivo do dicionário tem o formato esperado")

PROCESSAMENTO CONCLUÍDO COM SUCESSO
Total de atributos processados: 52
Atributos com descrição encontrada: 33
Atributos criados/sem descrição: 19

Primeiras linhas do resultado:
Atributo                                                      Descrição                  Valores Únicos  Qtd Valores Únicos
   V0001                                         Atributo Criado: V0001 21, 22, 23, 24, 25... (+8 mais)                  13
   V0026                                    Tipo de situação censitária                            1, 2                   2
   V0031                                                   Tipo de área                      1, 2, 3, 4                   4
    B001 O seu domicílio está cadastrado na unidade de saúde da família                   2.0, 1.0, 3.0                   3
    C006                                                           Sexo                        2.0, 1.0                   2

Arquivo salvo em: ../Bases/Criados/resumo_atributos.csv
